In [1]:
import requests
from bs4 import BeautifulSoup
import time

import numpy as np
#import matplotlib.pyplot as plt
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
#import pandas_datareader.data as web
from datetime import datetime, timedelta,date
import os
from openpyxl import load_workbook
import datetime


def get_monthly_report(company_id):
#https://goodinfo.tw/StockInfo/ShowSaleMonChart.asp?STOCK_ID=1101&STEP=DATA&CHT_CAT=ALL
    URL = 'https://goodinfo.tw/StockInfo/ShowSaleMonChart.asp?STOCK_ID='+company_id[:company_id.find(' ')]+'&STEP=DATA&CHT_CAT=ALL'
    my_headers = {'referer': 'https://goodinfo.tw/StockInfo/ShowSaleMonChart.asp?STOCK_ID='+company_id[:company_id.find(' ')],
                  'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36'}

    try:
        response = requests.get(URL, headers=my_headers)
        time.sleep(1)
    except:
        time.sleep(30)
        response = requests.get(URL, headers=my_headers)
        time.sleep(1)

    # print(response.status_code)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')


    dataStart = False
    data = []
    data.append([])
    data_index=0
    trs = soup.find_all('tr')


    for i in range(0,len(trs)):

        tds = trs[i].find_all('td')
        if (tds[0].get_text().find('/')!=-1):
            a=tds[0].get_text()
            year=int(a[:a.find('/')])
            month=int(a[a.find('/')+1:])
            month=month+1                #shift month by 1, to be compapible of finlab database
            if month ==13:
                month=1
                year=year+1
            #format_str = '%Y/%m'  # The format
            #datetime_obj = datetime.datetime.strptime(tds[0].get_text(), format_str)
            #print(datetime_obj)
            #dayCode.append(datetime_obj.date())
            
            data[data_index].append(datetime.datetime(year,month,10))  #date
            data[data_index].append(str(int(float(tds[7].get_text().replace(',',''))*100000)))   #當月營收
            data[data_index].append(tds[8].get_text().replace('+',''))                          #上月比較增減(%)
            data[data_index].append(tds[9].get_text().replace('+',''))                          #去年同月增減(%)
            data[data_index].append(str(int(float(tds[10].get_text().replace(',',''))*100000)))  #當月累計營收
            data[data_index].append(tds[11].get_text().replace('+',''))                          #前期比較增減(%)
            
    
            data.append([])
            data_index=data_index+1
            
  

    df = pd.DataFrame(data,columns=['date','當月營收','上月比較增減(%)','去年同月增減(%)','當月累計營收','前期比較增減(%)'])
    df['stock_id']=company_id
    df = df.set_index(['stock_id','date'])
    df = df.dropna(how='all', axis=0)#.dropna(how='all', axis=1)
    df=df.sort_index(level='date', ascending=True)
    print(df)
    #df.to_excel(writer, sheet_name='經營績效', header=False, index=False)





    return
stock='1101 台泥'
get_monthly_report(stock)

                         當月營收 上月比較增減(%) 去年同月增減(%)     當月累計營收 前期比較增減(%)
stock_id date                                                         
1101 台泥  1997-07-10   1082000         -         -    7633000         -
         1997-08-10   1297000      19.8         -    8930000         -
         1997-09-10   1062000     -18.1         -    9992000         -
         1997-10-10   1286000        21         -   11280000         -
         1997-11-10   1407000      9.43         -   12680000         -
         1997-12-10   1421000         1         -   14110000         -
         1998-01-10   1604000      12.9         -   15710000         -
         1998-02-10   1355000     -15.5         -    1355000         -
         1998-03-10    983000     -27.5         -    2338000         -
         1998-04-10   1492000      51.9         -    3829999         -
         1998-05-10   1484000     -0.55         -    5315000         -
         1998-06-10   1521000      2.49         -    6836000         -
      

In [5]:
df = pd.read_pickle("history/tables/monthly_report_new.pkl")

print(df[df.index.get_level_values('stock_id')=="1101 台泥"])
#print(df)

                         當月營收      上月營收    去年當月營收 上月比較增減(%) 去年同月增減(%)  \
stock_id date                                                           
1101 台泥  1997-07-10   1082000       NaN       NaN       NaN       NaN   
         1997-08-10   1297000       NaN       NaN      19.8       NaN   
         1997-09-10   1062000       NaN       NaN     -18.1       NaN   
         1997-10-10   1286000       NaN       NaN        21       NaN   
         1997-11-10   1407000       NaN       NaN      9.43       NaN   
         1997-12-10   1421000       NaN       NaN         1       NaN   
         1998-01-10   1604000       NaN       NaN      12.9       NaN   
         1998-02-10   1355000       NaN       NaN     -15.5       NaN   
         1998-03-10    983000       NaN       NaN     -27.5       NaN   
         1998-04-10   1492000       NaN       NaN      51.9       NaN   
         1998-05-10   1484000       NaN       NaN     -0.55       NaN   
         1998-06-10   1521000       NaN       NaN  

In [20]:
int(2747000.0)

2747000

In [22]:
#creating all stock list

#allID = df.index.get_level_values('stock_id')
#allID = list(dict.fromkeys(allID))  # remove duplicates
#writer = ExcelWriter('allstocklistMonthlyReport.xlsx', engine='openpyxl')
#df1 = pd.DataFrame(allID)
#df1.to_excel(writer, sheet_name='stock', header=False, index=False)
#writer.save()